# Introduction
For the social media challenges, we've harvested the tweets of about 60 NGOs and a number of hashtags. Additionally, 50 of those NGOs also have YouTube channels, where we harvested all channel and video comments.

The results of these efforts have been stored as Hive tables. The following gives an overview of the tables that are available for the social media team:

-----------------------------------------------------------------------------
Table              Contents
------------------ ----------------------------------------------------------
`twitter`          One row for each tweet, 73 variables describing the tweet.
`youtube_channels` One row for each channel comment in 15 variables.
`youtube_videos`   One row for each video in 16 variables.
`youtube_comments` One row for each comment in 16 variables
`ngos`             One row per targeted NGO
-----------------------------------------------------------------------------

You can find a more detailed description of the available fields in each table at the [Social Media data dictionary](https://github.com/MichalMilkowski1989/Datathon-Vienna-2018/wiki/Social-Media-Data-Dictionary).

# Data access

All data is stored in tables in the `default` Hive database. From within pyspark, it's easiest to access the data
using Spark's interface to Hive. Follow these steps to get a data set you can work with:

1. Write a SQL statement to retrieve the data you need
2. Use that (subset of the) data directly in Spark
3. Collect the data (or ideally, just some results) onto the Edge node.

Find some code examples below for each of the steps.

## Registering data (sub) sets as Spark DataFrames

In [1]:
yt_chan_sdf = spark.sql("SELECT * FROM test.youtube_channels")
twitter_sdf = spark.sql("SELECT * FROM test.twitter")

## Work with the Spark DataFrame directly in Spark

In [2]:
# Average retweet count per language
twitter_sdf. \
    groupby("lang"). \
    mean("retweet_count"). \
    head(5)

[Row(lang=u'en', avg(retweet_count)=2.8168314059306137),
 Row(lang=u'vi', avg(retweet_count)=0.0),
 Row(lang=u'ne', avg(retweet_count)=1.6666666666666667),
 Row(lang=u'ps', avg(retweet_count)=0.0),
 Row(lang=u'ro', avg(retweet_count)=1.0726256983240223)]

In [3]:
# Testing the hypothesis that quoting popular tweets increases your own retweet count

# Step (1): Preparing the data
model_data_sdf = twitter_sdf. \
    select("retweet_count", "quoted_retweet_count"). \
    dropna(). \
    filter("retweet_count > 0"). \
    filter("quoted_retweet_count > 0")

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["quoted_retweet_count"],
    outputCol="features")

output_sdf = assembler.transform(model_data_sdf)

# Step (2): Setting up the Poisson regression model
from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression(family="poisson",
                                  maxIter=10,
                                  regParam=0.3, 
                                  labelCol="retweet_count")

# Step (3): Fitting model to the prepared data
model = glr.fit(output_sdf)

In [4]:
# Step (4): Printing model results
summary = model.summary
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
print("Deviance Residuals: ")
summary.residuals().show()

Coefficient Standard Errors: [8.898054944901488e-07, 0.005564436782637518]
T Values: [2.6492929007781276, 327.00062731599166]
P Values: [0.008066039153730387, 0.0]
Dispersion: 1.0
Null Deviance: 85621.0527644
Residual Degree Of Freedom Null: 5327
Deviance: 85614.7652644
Residual Degree Of Freedom: 5326
AIC: 100838.034135
Deviance Residuals: 
+--------------------+
|   devianceResiduals|
+--------------------+
| -1.4186988015892787|
|  -2.588322309177586|
| -1.9597608321470814|
|   -0.93437826201872|
| -1.4187093338409071|
| -0.4869455859380116|
| -1.4187093338409071|
|  -2.588359973426336|
|  -2.588496508839914|
| -1.4187040677127842|
|  0.7039804059203871|
|  0.3155485803667238|
| -2.5883646814784984|
|   0.703802741562515|
|  -2.593009099987482|
|  2.3899387192104977|
| -1.9578879050518856|
|  0.3243690982261497|
|-0.48705313791336097|
| -0.4871720130823864|
+--------------------+
only showing top 20 rows



## Collect data/results to the edge node

In [5]:
import pandas as pd

yt_chan_local = yt_chan_sdf.toPandas()
yt_chan_local

,topic,country,ngo,authorDisplayName,authorProfileImageUrl,authorChannelUrl,authorChannelId_value,channelId,textDisplay,textOriginal,canRate,viewerRating,likeCount,publishedAt,updatedAt
0,migration,pl,UC_ntaIWaLOutp-eUhcDrp8A,Mateo Holding bvba,https://yt3.ggpht.com/-4xZL6RZDWsw/AAAAAAAAAAI...,http://www.youtube.com/channel/UCuiSIU9Eu2gvf-...,UCuiSIU9Eu2gvf-zfVtLP-lw,UC_ntaIWaLOutp-eUhcDrp8A,Banda faszystowskich świń !!!,Banda faszystowskich świń !!!,True,none,1,2018-01-21 09:18:47,2018-01-21 09:18:47
1,migration,pl,UC_ntaIWaLOutp-eUhcDrp8A,przemek fracisz,https://yt3.ggpht.com/--t_NZY3SFaI/AAAAAAAAAAI...,http://www.youtube.com/channel/UCGzRTeP-KGgHcq...,UCGzRTeP-KGgHcqwqS5S4uzA,UC_ntaIWaLOutp-eUhcDrp8A,dobry nacjonalista to martwy nacjonalista <br ...,dobry nacjonalista to martwy nacjonalista nas...,True,none,1,2017-07-14 21:45:22,2017-07-14 21:45:22
2,migration,pl,UC_ntaIWaLOutp-eUhcDrp8A,Zoroark IllusionMaster,https://yt3.ggpht.com/-aT6yQc-Bi_w/AAAAAAAAAAI...,http://www.youtube.com/channel/UCEo-KBBVXdHYq3...,UCEo-KBBVXdHYq3BJvFDH4Sw,UC_ntaIWaLOutp-eUhcDrp8A,ONR = Bradctwo NOD z seri C&amp;C,ONR = Bradctwo NOD z seri C&C,True,none,1,2017-06-05 06:21:53,2017-06-05 06:21:53
3,migration,pl,UC_ntaIWaLOutp-eUhcDrp8A,ROTA - ENDECY,https://yt3.ggpht.com/-mJt613_9D0M/AAAAAAAAAAI...,http://www.youtube.com/channel/UChegzqbPuIeceb...,UChegzqbPuIecebGcHeAtKNg,UC_ntaIWaLOutp-eUhcDrp8A,PYTANIE PUBLICZNE DO MW I ONR: CHŁOPCY Z MW I ...,PYTANIE PUBLICZNE DO MW I ONR: CHŁOPCY Z MW I ...,True,none,1,2017-05-30 09:00:03,2017-05-30 09:00:03
4,migration,pl,UC_ntaIWaLOutp-eUhcDrp8A,Karol Kowalski,https://yt3.ggpht.com/-tSwdrWFW72M/AAAAAAAAAAI...,http://www.youtube.com/channel/UCzBz38u14DYVnE...,UCzBz38u14DYVnEd0-UiZugA,UC_ntaIWaLOutp-eUhcDrp8A,CWP. Wielki Szacunek dla was.,CWP. Wielki Szacunek dla was.,True,none,1,2017-05-15 17:07:12,2017-05-15 17:07:12
5,migration,pl,UC_ntaIWaLOutp-eUhcDrp8A,Ekstrim,https://yt3.ggpht.com/-kOjI6sRsXok/AAAAAAAAAAI...,http://www.youtube.com/channel/UCj18C4o_UE81IH...,UCj18C4o_UE81IHiLpUonkdQ,UC_ntaIWaLOutp-eUhcDrp8A,Chuj wam w dupsko,Chuj wam w dupsko,True,none,0,2017-05-13 12:21:16,2017-05-13 12:21:16
6,migration,pl,UC_ntaIWaLOutp-eUhcDrp8A,georgiors sr,https://yt3.ggpht.com/-_g-wkgBSevw/AAAAAAAAAAI...,http://www.youtube.com/channel/UCWBv514L3ypvo2...,UCWBv514L3ypvo2AitziH9cw,UC_ntaIWaLOutp-eUhcDrp8A,ROBICIE KAWAŁ DOBREJ ROBOTY,ROBICIE KAWAŁ DOBREJ ROBOTY,True,none,4,2017-03-25 21:46:21,2017-03-25 21:46:21
7,migration,pl,UC_ntaIWaLOutp-eUhcDrp8A,Dariusz Piszczek,https://yt3.ggpht.com/-4Cak_yeUmcg/AAAAAAAAAAI...,http://www.youtube.com/channel/UCKyfIX_OH6kcZt...,UCKyfIX_OH6kcZt7OWc-wKUg,UC_ntaIWaLOutp-eUhcDrp8A,Apeluje o konsolidacje srodowisk Narodowych !!...,Apeluje o konsolidacje srodowisk Narodowych !!...,True,none,2,2017-01-03 23:27:03,2017-01-03 23:27:03
8,migration,pl,UC_ntaIWaLOutp-eUhcDrp8A,kadykianus,https://yt3.ggpht.com/-Nvk60XrwaZE/AAAAAAAAAAI...,http://www.youtube.com/channel/UCsFK_N0TS3ctkY...,UCsFK_N0TS3ctkY9noXVSe5g,UC_ntaIWaLOutp-eUhcDrp8A,"Uważam, że jest to bardzo sympatyczny portal i...","Uważam, że jest to bardzo sympatyczny portal i...",True,none,2,2016-10-29 15:02:28,2016-10-29 15:02:28
9,migration,at,UC_MNgEkOK_crUltwOlTiCCA,ThomasAir,https://yt3.ggpht.com/-c2HhsrclCQc/AAAAAAAAAAI...,http://www.youtube.com/channel/UCfdLkkEnxUV4eM...,UCfdLkkEnxUV4eMBDiEEgk2g,UC_MNgEkOK_crUltwOlTiCCA,GEZ finanziert?,GEZ finanziert?,True,none,0,2016-12-11 11:41:08,2016-12-11 11:41:08
